In [1]:
import pandas as pd

# Load data
inventory_df = pd.read_csv('inventory_log.csv')
sales_df = pd.read_csv('sales_log.csv')
returns_df = pd.read_csv('returns_log.csv')

# Quick view
print("Inventory:")
print(inventory_df.head())

print("\nSales:")
print(sales_df.head())

print("\nReturns:")
print(returns_df.head())


Inventory:
         date product_id  stock_in  stock_out  stock_remaining
0  2023-01-01       P001        12          6                4
1  2023-01-01       P002        11          1                9
2  2023-01-01       P003        11          7                3
3  2023-01-01       P004        12          4                6
4  2023-01-01       P005        12          5                5

Sales:
         date product_id  quantity_sold  unit_price
0  2023-01-01       P001              6       62.43
1  2023-01-01       P002              5       92.43
2  2023-01-01       P003              8       10.92
3  2023-01-01       P004              5       45.19
4  2023-01-01       P005              3       82.83

Returns:
         date product_id  quantity_returned          return_reason
0  2023-01-01       P009                  1             Wrong item
1  2023-01-01       P001                  1             Wrong item
2  2023-01-01       P018                  1  Customer changed mind
3  2023-01-02

In [2]:
print(inventory_df.info())
print(sales_df.info())
print(returns_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   date             1800 non-null   object
 1   product_id       1800 non-null   object
 2   stock_in         1800 non-null   int64 
 3   stock_out        1800 non-null   int64 
 4   stock_remaining  1800 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 70.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           1800 non-null   object 
 1   product_id     1800 non-null   object 
 2   quantity_sold  1800 non-null   int64  
 3   unit_price     1800 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 56.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns 

In [5]:
# Step 3: Aggregate totals and calculate shrinkage

# Group inventory data: sum stock_in, stock_out and get last stock_remaining
inventory_total = inventory_df.groupby('product_id').agg({
    'stock_in': 'sum',
    'stock_out': 'sum',
    'stock_remaining': 'last'
}).reset_index()

# Group sales data: sum quantity_sold by product
sales_total = sales_df.groupby('product_id')['quantity_sold'].sum().reset_index()

# Group returns data: sum quantity_returned by product
returns_total = returns_df.groupby('product_id')['quantity_returned'].sum().reset_index()

# Merge all grouped dataframes on product_id
df = inventory_total.merge(sales_total, on='product_id', how='left')
df = df.merge(returns_total, on='product_id', how='left')

# Replace NaN values with 0 (for products without sales or returns)
df.fillna(0, inplace=True)

# Calculate shrinkage:
# Shrinkage = stock_in - (quantity_sold + quantity_returned + stock_remaining)
df['shrinkage'] = df['stock_in'] - (df['quantity_sold'] + df['quantity_returned'] + df['stock_remaining'])

# Sort by shrinkage descending to find products with highest shrinkage
df = df.sort_values(by='shrinkage', ascending=False)

# Show top 10 products with highest shrinkage
print(df.head(10))


   product_id  stock_in  stock_out  stock_remaining  quantity_sold  \
10       P011       934        704                8            509   
12       P013       938        733                0            528   
16       P017       925        690                0            536   
19       P020       915        699                7            521   
1        P002       944        721                8            549   
14       P015       955        686                8            582   
11       P012       897        743                9            523   
15       P016       940        746                4            578   
13       P014       868        711               11            496   
8        P009       906        691                9            538   

    quantity_returned  shrinkage  
10                 17        400  
12                 14        396  
16                 15        374  
19                 13        374  
1                  14        373  
14                 

In [13]:
import plotly.express as px

# Take top 10 products by shrinkage
top_shrink = df.sort_values(by='shrinkage', ascending=False).head(10)

fig = px.bar(
    top_shrink,
    x='product_id',
    y='shrinkage',
    title='Top 10 Products by Inventory Shrinkage',
    labels={'product_id': 'Product ID', 'shrinkage': 'Units Lost (Shrinkage)'},
    color='shrinkage',
    color_continuous_scale='Reds'
)

fig.show()


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Prepare features and target
X = df[['stock_in', 'quantity_sold', 'quantity_returned']]
y = df['shrinkage']

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate model
print("Model R² score:", model.score(X_test, y_test))


Model R² score: 0.9895746689563556


In [12]:
import plotly.express as px
import pandas as pd

# Predict on test set
y_pred = model.predict(X_test)
residuals = y_test - y_pred

# Create a DataFrame for Plotly
residuals_df = pd.DataFrame({
    'Predicted Shrinkage': y_pred,
    'Residuals': residuals
})

# 1) Residuals Distribution (Histogram + KDE)
fig1 = px.histogram(residuals_df, x='Residuals', nbins=50, marginal="rug",
                    title='Residuals Distribution', opacity=0.75)
fig1.show()

# 2) Residuals vs Predicted Scatter Plot
fig2 = px.scatter(residuals_df, x='Predicted Shrinkage', y='Residuals',
                  title='Residuals vs Predicted Shrinkage',
                  labels={'Residuals': 'Residuals', 'Predicted Shrinkage': 'Predicted Shrinkage'},
                  trendline='lowess')
fig2.add_shape(type="line",
               x0=residuals_df['Predicted Shrinkage'].min(),
               y0=0,
               x1=residuals_df['Predicted Shrinkage'].max(),
               y1=0,
               line=dict(color="Red", dash="dash"))
fig2.show()


In [10]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score

# Random Forest
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
print("Random Forest R²:", r2_score(y_test, rf_pred))

# Gradient Boosting
gb = GradientBoostingRegressor(random_state=42)
gb.fit(X_train, y_train)
gb_pred = gb.predict(X_test)
print("Gradient Boosting R²:", r2_score(y_test, gb_pred))


Random Forest R²: 0.7687706838820284
Gradient Boosting R²: 0.8382298875721109


### Linear regression is better in compare dto both Random Forest and Gradient Boosting

In [16]:
import joblib
joblib.dump(model, 'linear_regression_shrinkage_model.joblib')


['linear_regression_shrinkage_model.joblib']

In [24]:
import pandas as pd
import joblib

# Load your trained model
model = joblib.load(r'A:\amazon\linear_regression_shrinkage_model.joblib')

# Load your new inventory data from CSV
new_inventory = pd.read_csv(r'A:\amazon\new_inventory_month.csv')

# Check columns to make sure required features exist
print("Columns in new data:", new_inventory.columns)

# Select the features exactly as used during training
# Assuming your model expects these columns: 'stock_in', 'quantity_sold', 'quantity_returned'
X_new = new_inventory[['stock_in', 'quantity_sold', 'quantity_returned']]

# Predict shrinkage
new_inventory['predicted_shrinkage'] = model.predict(X_new)

# Show predictions
print(new_inventory[['product_id', 'predicted_shrinkage']])

# Save predictions to a CSV file
new_inventory.to_csv(r'A:\amazon\new_inventory_shrinkage_predictions.csv', index=False)


Columns in new data: Index(['product_id', 'stock_in', 'quantity_sold', 'quantity_returned'], dtype='object')
  product_id  predicted_shrinkage
0       P001            32.595962
1       P002            19.432064
2       P003            13.005724
3       P004            16.353570
4       P005            23.163338
5       P006            23.888328
6       P007            13.730714
7       P008            16.805332
8       P009            18.532417
9       P010            15.767132


In [25]:
import pandas as pd
import plotly.express as px

# Assuming new_inventory already has the 'predicted_shrinkage' column
# If not, load your CSV again
# new_inventory = pd.read_csv("A:/amazon/new_inventory_shrinkage_predictions.csv")

# Bar chart for predicted shrinkage by product
fig = px.bar(new_inventory, 
             x='product_id', 
             y='predicted_shrinkage', 
             title='Predicted Shrinkage per Product',
             labels={'predicted_shrinkage': 'Predicted Units Lost', 'product_id': 'Product ID'},
             text='predicted_shrinkage')

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(yaxis=dict(range=[0, new_inventory['predicted_shrinkage'].max() * 1.2]))

fig.show()


In [26]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display, HTML

# Load your data (adjust path)
df = pd.read_csv('new_inventory_shrinkage_predictions.csv')

# KPIs
total_products = df['product_id'].nunique()
total_predicted_shrinkage = df['predicted_shrinkage'].sum()
avg_shrinkage = df['predicted_shrinkage'].mean()

# Dashboard title
display(HTML("<h1 style='text-align:center;'>Inventory Shrinkage Business Dashboard</h1>"))

# Display KPIs
print(f"Total Products: {total_products}")
print(f"Total Predicted Shrinkage Units: {total_predicted_shrinkage:.2f}")
print(f"Average Shrinkage per Product: {avg_shrinkage:.2f}")

# Bar chart for top 10 products by shrinkage
top_shrink = df.sort_values(by='predicted_shrinkage', ascending=False).head(10)

fig = go.Figure(
    data=[go.Bar(x=top_shrink['product_id'], y=top_shrink['predicted_shrinkage'], marker_color='crimson')],
    layout_title_text="Top 10 Products by Predicted Shrinkage"
)
fig.update_layout(
    xaxis_title="Product ID",
    yaxis_title="Predicted Shrinkage Units",
    template="plotly_white"
)
fig.show()

# Optional: Pie chart showing shrinkage distribution
fig2 = go.Figure(
    data=[go.Pie(labels=df['product_id'], values=df['predicted_shrinkage'], hole=.3)]
)
fig2.update_layout(title="Shrinkage Distribution by Product")
fig2.show()


Total Products: 10
Total Predicted Shrinkage Units: 193.27
Average Shrinkage per Product: 19.33


In [27]:
# Assuming df is your data with predictions and sales info
total_sales = df['quantity_sold'].sum()
total_returns = df['quantity_returned'].sum()

print(f"Total Sales Units: {total_sales}")
print(f"Total Returns Units: {total_returns}")


Total Sales Units: 1525
Total Returns Units: 62


In [ ]:
import ipywidgets as widgets
from IPython.display import display

product_dropdown = widgets.Dropdown(
    options=df['product_id'].unique(),
    description='Product:',
    disabled=False,
)

output = widgets.Output()

def on_product_change(change):
    output.clear_output()
    product = change['new']
    data = df[df['product_id'] == product].iloc[0]
    with output:
        print(f"Product ID: {data['product_id']}")
        print(f"Predicted Shrinkage: {data['predicted_shrinkage']:.2f}")
        print(f"Quantity Sold: {data['quantity_sold']}")
        print(f"Quantity Returned: {data['quantity_returned']}")

product_dropdown.observe(on_product_change, names='value')

display(product_dropdown, output)

# Trigger initial display
product_dropdown.value = df['product_id'].iloc[0]


Dropdown(description='Product:', options=('P001', 'P002', 'P003', 'P004', 'P005', 'P006', 'P007', 'P008', 'P00…

Output()

In [30]:
!pip install plotly ipywidgets


Defaulting to user installation because normal site-packages is not writeable


In [31]:
import pandas as pd

# Load your shrinkage-predicted file
df = pd.read_csv("A:/amazon/new_inventory_shrinkage_predictions.csv")
df.head()


,product_id,stock_in,quantity_sold,quantity_returned,predicted_shrinkage
0,P001,150,120,5,32.595962
1,P002,180,160,10,19.432064
2,P003,200,190,8,13.005724
3,P004,140,130,3,16.353570
4,P005,170,150,6,23.163338


In [32]:
total_stock = df['stock_in'].sum()
total_sold = df['quantity_sold'].sum()
total_returned = df['quantity_returned'].sum()
total_shrinkage = df['predicted_shrinkage'].sum()

print(f"📦 Total Stock In: {total_stock}")
print(f"🛒 Total Sold: {total_sold}")
print(f"↩️ Total Returned: {total_returned}")
print(f"❗ Predicted Shrinkage: {total_shrinkage:.2f}")


📦 Total Stock In: 1685
🛒 Total Sold: 1525
↩️ Total Returned: 62
❗ Predicted Shrinkage: 193.27


In [36]:
import ipywidgets as widgets
from IPython.display import display

# Create labeled boxes for KPIs
kpi1 = widgets.HTML(f"<h3>📦 Stock In</h3><p>{total_stock}</p>")
kpi2 = widgets.HTML(f"<h3>🛒 Sold</h3><p>{total_sold}</p>")
kpi3 = widgets.HTML(f"<h3>↩️ Returns</h3><p>{total_returned}</p>")
kpi4 = widgets.HTML(f"<h3>❗ Shrinkage</h3><p>{total_shrinkage:.2f}</p>")

# Show KPIs in a row
display(widgets.HBox([kpi1, kpi2, kpi3, kpi4]))


In [37]:
import plotly.express as px

product_dropdown = widgets.Dropdown(
    options=df['product_id'].unique(),
    description='Product:',
    value=df['product_id'].iloc[0]
)

def plot_data(product_id):
    filtered = df[df['product_id'] == product_id]
    fig = px.bar(
        filtered.melt(id_vars='product_id', value_vars=['stock_in', 'quantity_sold', 'quantity_returned', 'predicted_shrinkage']),
        x='variable',
        y='value',
        title=f'📊 Inventory Overview for {product_id}',
        color='variable'
    )
    fig.show()

widgets.interact(plot_data, product_id=product_dropdown)


interactive(children=(Dropdown(description='Product:', options=('P001', 'P002', 'P003', 'P004', 'P005', 'P006'…

<function __main__.plot_data(product_id)>